# Change detection using M3C2 algorithm
With the help of the change detection algorithm M3C2 we can remove non-static objects from the point cloud. Resulting in a point cloud with only static objects, that we name obstacles. The provided example tile in the folder `../datasets/pointclouds/m3c2/` is generated from the M3C2 plugin inside CloudCompare. This notebook presents how the M3C2 algorithm can be run using the CloudComPy package.

In [ ]:
import os
# os.environ["_CCTRACE_"]="ON"  # Uncomment to show CloudCompare debug output

import cloudComPy as cc
import numpy as np
import pathlib
import re
from tqdm.notebook import tqdm # Requires tqdm and ipywidgets
import multiprocessing

In [ ]:
cc.initCC()  # to do once before using plugins or dealing with numpy

In [ ]:
pc_data_folder = '../datasets/pointclouds/'

# Resume previous incomplete run
resume = True

# The distance analysis is executed on a number of points of interest called *core points*. 
# This could be e.g. the entire reference point cloud, a downsampled version of it, an equistant grid etc. 
# In the m3c2_params configuration file, we choose the entire point cloud:
param_file = '../datasets/m3c2_params.txt'

In [ ]:
# Find the number of CPU cores available on the system
max_thread_count = multiprocessing.cpu_count()

# Read in the file
with open(param_file, 'r') as file :
    filedata = file.read()

# Replace the target string
filedata = filedata.replace('{cpu_count}', str(max_thread_count))

# Write the file out again
with open(param_file, 'w') as file:
    file.write(filedata)

In [ ]:
# Create folder for m3c2 files.
new_path = f'{pc_data_folder}m3c2'
pathlib.Path(new_path).mkdir(parents=True, exist_ok=True)

In [ ]:
# Get a list of all tilecodes for which we have two runs.
def get_tilecodes_from_folder(las_folder, las_prefix=''):
    """Get a set of unique tilecodes for the LAS files in a given folder."""
    files = pathlib.Path(las_folder).glob(f'{las_prefix}*.laz')
    tilecodes = set([re.match(r'.*(\d{4}_\d{4}).*', file.name)[1] for file in files])
    return tilecodes

all_tiles = (get_tilecodes_from_folder(f'{pc_data_folder}obstacles_run1/')
             .intersection(get_tilecodes_from_folder(f'{pc_data_folder}obstacles_run2/')))

if resume:
    done_tiles = get_tilecodes_from_folder(f'{pc_data_folder}m3c2/', las_prefix='m3c2')
    all_tiles = all_tiles - done_tiles

In [ ]:
tile_tqdm = tqdm(all_tiles, unit='tile', smoothing=0)

for tilecode in tile_tqdm:
    tile_tqdm.set_postfix_str(tilecode)
    
    cloud1 = cc.loadPointCloud(f'{pc_data_folder}obstacles_run1/obst_{tilecode}.laz', cc.CC_SHIFT_MODE.NO_GLOBAL_SHIFT)
    cloud2 = cc.loadPointCloud(f'{pc_data_folder}obstacles_run2/obst_{tilecode}.laz', cc.CC_SHIFT_MODE.NO_GLOBAL_SHIFT)
    
    if cc.isPluginM3C2():
        import cloudComPy.M3C2

        m3c2_cloud = cc.M3C2.computeM3C2([cloud1, cloud2], param_file)

        if m3c2_cloud is None:
            print(f'No M3C2 distances found for tile {tilecode}. Next...')
            continue # TODO handle
        if m3c2_cloud.getNumberOfScalarFields() < 3:
            raise RuntimeError
        dic = m3c2_cloud.getScalarFieldDic()
        # The calculated distances
        sf = m3c2_cloud.getScalarField(dic['M3C2 distance'])
        if sf is None:
            raise RuntimeError

        cc.SavePointCloud(m3c2_cloud, f'{pc_data_folder}m3c2/m3c2_{tilecode}.laz')
    else:
        print('M3C2 plugin not found.')
        break